In [1]:
!pip install --upgrade pip
!pip install xgboost
!pip install lightgbm
!pip install gcsfs
!pip install spacy
!pip install keras
!pip install textblob
!python3 -m spacy download en
!python3 -m spacy download en_core_web_md

Requirement already up-to-date: pip in /opt/conda/anaconda/lib/python3.6/site-packages (20.0.2)


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/conda/anaconda/lib/python3.6/site-packages/en_core_web_sm -->
/opt/conda/anaconda/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [2]:
from platform import python_version
print("python version: ", python_version())

python version:  3.6.10


In [3]:
%%time
import os, sys, time, json, re, string, datetime 
import multiprocessing as mp
import pandas as pd
import numpy as np
import spacy
from google.cloud import storage

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer, Normalizer, normalize
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction import  DictVectorizer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline, FeatureUnion

import seaborn as sns
import math

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
import sklearn.tree as skt

from xgboost import XGBClassifier
from xgboost.core     import XGBoostError
import xgboost as xgb

from IPython import display

from lightgbm import LGBMClassifier
from lightgbm.sklearn import LightGBMError

/opt/conda/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/opt/conda/anaconda/lib/python3.6/site-packages/distributed/config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


CPU times: user 877 ms, sys: 99 ms, total: 976 ms
Wall time: 973 ms


In [4]:
%%time
df_train = pd.read_csv('gs://dataproc-6ca41800-27b4-47d5-abee-55c011dfa389-asia-southeast1/data/kaggle/train.csv')
df_train.info()
df_train['Outcome'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44183 entries, 0 to 44182
Data columns (total 3 columns):
Comment    44183 non-null object
Outcome    44183 non-null int64
Id         44183 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.0+ MB
CPU times: user 449 ms, sys: 55 ms, total: 504 ms
Wall time: 1.08 s


1    24992
0    19191
Name: Outcome, dtype: int64

In [5]:
import en_core_web_sm

class TextFeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.nlp = en_core_web_sm.load()
        self.future_words = ["tomorrow", "future", "futures"]
        self.past_words = ["yesterday", "past","pasts"]
        self.first_view_words = ["I", "my", "mine"]
        self.second_view_words = ["you", "your", "yours"]
        self.third_view_words = ["they", "their", "them"]
        self.it_view_words = ["it", "its"]
        
    def fit(self, x, y=None):
        return self
    
    def count_pronouns(doc):
        segment = doc.text.lower().split()
        counter = {"1sg": 0, "1pl": 0}
        for pronoun in FIRST_SINGULAR:
            counter["1sg"] += segment.count(pronoun)
        for pronoun in FIRST_PLURAL:
            counter["1pl"] += segment.count(pronoun)
        return counter

    def get_n_future_oriented_words(self, doc):
        will_aux = [t for t in doc if t.tag_ == "MD" and t.lower_ in {"will", "wo", "shall", "sha"}]
        going_to = [t for t in doc if t.dep_ == "xcomp" and t.head.lemma_ == "go"]
        other_future_words = [t for t in doc if t.lower_ in self.future_words]
        return len(will_aux) + len(going_to) + len(other_future_words)

    def get_n_past_oriented_words(self, doc):
        would_aux = [t for t in doc if t.tag_ == "MD" and t.lower_ in {"would", "wouldn't", "should", "shouldn't"}]
        went_to = [t for t in doc if t.dep_ == "xcomp" and t.head.lemma_ == "went"]
        other_past_words = [t for t in doc if t.lower_ in self.past_words]
        return len(would_aux) + len(went_to) + len(other_past_words)
    
    def get_n_view_words(self, doc, list):
        n_first_view_words = [t for t in doc if t.lower_ in self.first_view_words]
        return len(n_first_view_words) 

    def get_n_words_before_main_verb(self,doc):
        numbers = [0]
        for sent in doc.sents:
            main = [t for t in sent if t.dep_ == "ROOT"][0]
            if main.pos_ == "VERB":
                dist_to_init = main.i - sent[0].i
                numbers.append(dist_to_init)
        return np.mean(numbers)

    def get_n_complex_clauses(self,doc):
        embedded_elements_count = []
        for sent in doc.sents:
            n_embedded = len(
                [t for t in sent if t.dep_ in {"ccomp", "xcomp", "advcl", "dative"}]
            )
            embedded_elements_count.append(n_embedded)
        return np.mean(embedded_elements_count)
    
    def get_n_punctuation(self,doc):
        word_count = len(doc)
        unique_word_count = len(set(doc))
        char_count = 0 
        dict_punct_count = {}
        for c in doc:
            if str(c) in string.punctuation:
                dict_punct_count[str(c)] = dict_punct_count.get(str(c), 0) + 1
            else: 
                char_count += 1
        
        punctuation_count = sum(dict_punct_count.values())        
        feature_dict = { 
                "n_word_count": word_count, 
                "n_char_count": char_count, 
                "n_char_count_percentage": char_count / (word_count + 1e-10),  
                "n_unique_word_count": unique_word_count, 
                "n_unique_word_count_percentage": unique_word_count / (word_count + 1e-10),
                "n_punctuation_count": punctuation_count, 
                "n_punctuation_count_percentage": punctuation_count / (word_count + 1e-10)}
       
        puntuation_list = ["!", "?", "=", "{", "<", ">", "(", "+", "-", "*", "/", "[","#"]
        puntuation_dict={}        
        feature_dicts = []
        i = 0
        for key in dict_punct_count: 
            value = 0.0
            dict_name = "n_punctuation_count_" + str(i)
            dict_name_precentage = dict_name + "_p"
            for p in puntuation_list:
                if key == p:
                    value = dict_punct_count.get(key, 0)
                    
            p_dict = {dict_name: value, dict_name_precentage: value / (word_count + 1e-10)}
            puntuation_dict.update(p_dict)
            i = i + 1
        
        return puntuation_dict  
    
    
    def get_n_pos(self,doc):
        word_count = len(doc)
        pos_family = {'noun' : ['NN','NNS','NNP','NNPS'],
                      'pron' : ['PRP','PRP$','WP','WP$'],
                      'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
                      'adj' :  ['JJ','JJR','JJS'],
                      'adv' : ['RB','RBR','RBS','WRB']}
        dict = {'noun_count' : 0, 'noun_p' : 0,
                'pronn_count' : 0, 'pronn_p' : 0,
                'verbn_count' : 0, 'verbn_p' : 0,
                'adjn_count' :  0, 'adjn_p' :  0,
                'advn_count' : 0, 'advn_p' : 0}
        for key in pos_family: 
            cnt = 0
            for flag in pos_family.get(key,0):
                try:
                    tags = textblob.TextBlob(doc).tags
                    for tup in tags:
                        ppo = list(tup)[1]
                        if ppo in pos_family[flag]:
                            cnt += 1
                except:
                    pass   
            dict.update({key + '_count': cnt,key + '_p':cnt/word_count}) 
        return dict     
    
    def transform(self, data):
        feature_dicts = []
        docs = self.nlp.pipe(data)
        for doc in docs:
            doc_len = len(doc)
            n_past_oriented_words = self.get_n_past_oriented_words(doc)
            n_future_oriented_words = self.get_n_future_oriented_words(doc)
            n_words_before_main_verb = self.get_n_words_before_main_verb(doc)
            n_complex_clauses = self.get_n_complex_clauses(doc)
            n_first_view_words = self.get_n_view_words(doc, self.first_view_words)
            n_second_view_words = self.get_n_view_words(doc, self.second_view_words)
            n_third_view_words = self.get_n_view_words(doc, self.third_view_words)
            n_it_view_words = self.get_n_view_words(doc, self.it_view_words)            
            feature_dict = {
                "n_past_oriented_words": n_past_oriented_words,
                "p_past_oriented_words": n_past_oriented_words / doc_len,
                "n_future_oriented_words": n_future_oriented_words,
                "p_future_oriented_words": n_future_oriented_words / doc_len,
                "n_complex_clauses": n_complex_clauses,
                "p_complex_clauses": n_complex_clauses / doc_len,
                "n_words_before_main_verb": n_words_before_main_verb,
                "p_words_before_main_verb": n_words_before_main_verb / doc_len,
                "n_complex_clauses": n_complex_clauses,
                "p_complex_clauses": n_complex_clauses / doc_len,
                "n_first_view_words": n_first_view_words,
                "p_first_view_words": n_first_view_words / doc_len,
                "n_second_view_words": n_second_view_words,
                "p_second_view_words": n_second_view_words / doc_len,
                "n_third_view_words": n_third_view_words,
                "p_third_view_words": n_third_view_words / doc_len,
                "n_it_view_words": n_it_view_words,
                "p_it_view_words": n_it_view_words / doc_len   
            }
            puntuation_dict = self.get_n_punctuation(doc)
            feature_dict.update(puntuation_dict)            
            pos_dict = self.get_n_pos(doc)
            feature_dict.update(pos_dict)
            
            feature_dicts.append(feature_dict)
        return feature_dicts

In [6]:
%%time
def build_pipeline():
    base_pipeline = Pipeline(
        steps=[
            ("tfidf", TfidfVectorizer())
        ]
    )   
    word_pipeline = Pipeline(
        steps=[
            ("word_stats", TextFeatureTransformer()),
            ("word_dict_vect", DictVectorizer())
        ]
    )
    combined_features = FeatureUnion(
        transformer_list=[
            ("base", base_pipeline),
            ("word", word_pipeline)
        ]
    )  
    return combined_features

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs


In [7]:
def build_models(): 
    clf_logis = LogisticRegression(random_state=100, solver='lbfgs', multi_class='multinomial',max_iter=500, n_jobs=-1)
    
    clf_dt = DecisionTreeClassifier(max_depth=5, random_state=100)
 
    clf_gbt= GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, random_state=100)
   
    clf_gnb = GaussianNB(priors = None)
          
    clf_rfc = RandomForestClassifier(n_estimators = 50, max_depth = 5)
    
    clf_knn = KNeighborsClassifier(n_neighbors = 5, algorithm = 'auto', leaf_size = 30)
   
    clf_lgbm= LGBMClassifier(n_jobs=-1, random_state=100, 
                             subsample=0.3, num_trees=200, 
                             n_estimators=100, min_child_weight=0.01, 
                             max_depth=1, learning_rate=0.1, 
                             gamma=1, colsample_bytree=0.6)
    
    clf_xgb = XGBClassifier(objective='binary:logistic', random_state=100,
                            subsample=1, num_trees=200, 
                            n_estimators=500, min_child_weight=0.01, 
                            max_depth=1, learning_rate=0.1, 
                            gamma=0.01, colsample_bytree=0.3)
    
    param_tune = {'max_depth': range(1, 3, 5),
                  'min_child_weight': [0.01, 0.1, 1],
                  'gamma':[1, 0.1, 0.01],
                  'subsample':[0.3, 0.6, 1],
                  'colsample_bytree':[0.3, 0.6, 1],
                  'num_trees':[100, 200, 300],
                  'learning_rate':[0.01, 0.1],
                  'n_estimators': [100,300,500]}
    
    clf_logis_param_tune = {'penalty':['l2'], 
                            'C':[0.01,0.05,0.1,0.5,1,5,10,50,100], 
                            'solver':['lbfgs'],
                            'multi_class':['ovr','multinomial']}
   
    ##[model_name, model, parameter_gride, run_flag, tuning_flag, probability_col]
    models = [["LogisticRegression", clf_logis, clf_logis_param_tune, True, False],
              ["DecisionTreeClassifier",clf_dt, param_tune, True, False],              
              ["GaussianNB",clf_gnb, param_tune, False, False],
              ["RandomForestClassifier", clf_rfc, param_tune, True, False],
              ["KNeighborsClassifier" , clf_knn, param_tune, True, False],
              ["LGBMClassifier" , clf_lgbm, param_tune, True, False],
              ["XGBClassifier" , clf_xgb, param_tune, True, False]]
    return models

In [8]:
def run_models(models, X_train, y_train, X_test, y_test): 
    df_outcome = pd.DataFrame(columns=['model_name', 'model_object', 'cv_train', 'train_accuray_score', 'test_accuray_score'])      
    kfold= StratifiedKFold(n_splits=5, random_state=100) 
    for item in models:        
        name = item[0]
        model = item[1]
        param_tune = item[2]
        run_flag = item[3]
        tuning_flag = item[4]       
        if run_flag:                
            print("**********Run Model {0}**********".format(name))    
            t = time.time()
            if tuning_flag: 
                model = RandomizedSearchCV(estimator = model, param_distributions = param_tune,
                                           scoring = "accuracy", n_jobs = -1, cv = 5, verbose=0)
                model.fit(X_train, y_train) 
                print("{0} Best Parameters: {1}".format(name, model.best_params_))
            else:
                model.fit(X_train, y_train)            
            cv_results = cross_val_score(model, X_train, y_train, cv=kfold).mean()    
            train_score = metrics.accuracy_score(model.predict(X_train), y_train)
            test_score = metrics.accuracy_score(model.predict(X_test), y_test)
            vals = [[name, model, cv_results, train_score, test_score]]
            df2 = pd.DataFrame(vals, columns=df_outcome.columns)
            df_outcome = df_outcome.append(df2)  
            print("{0} -  train_score: {1}, test_score:{2}".format(name, train_score, test_score))
            print("time taken for {0}: {1}".format(name, str(datetime.timedelta(seconds=time.time() - t))))
    
    return df_outcome

In [ ]:
%%time
ms = time.time()
small_smaple_flag= False
sample_df_train = df_train.sample(frac=0.01, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(sample_df_train.Comment, sample_df_train.Outcome, test_size=0.2)
if small_smaple_flag == False:
    X_train, X_test, y_train, y_test = train_test_split(df_train.Comment, df_train.Outcome, test_size=0.2)

pipeline = build_pipeline().fit(X_train)
print("time taken for fit.{0}".format(time.time() - ms))
ms = time.time()
X_train_transformated = pipeline.transform(X_train)
print("time taken for X_train_transformated.{0}".format(time.time() - ms))
ms = time.time()
X_test_transformated = pipeline.transform(X_test)
print("time taken for X_train_transformated.{0}".format(time.time() - ms))

time taken for fit.602.51136302948
time taken for X_train_transformated.587.1272983551025
time taken for X_train_transformated.148.87236094474792
CPU times: user 19min 20s, sys: 2min 58s, total: 22min 18s
Wall time: 22min 18s


In [ ]:
%%time
models = build_models() 
df_outcome = run_models(models, X_train_transformated, y_train, X_test_transformated, y_test)
df_outcome = df_outcome.sort_values(by=['test_accuray_score'], ascending=False)
display.display(df_outcome)

**********Run Model LogisticRegression**********
LogisticRegression -  train_score: 0.8071634696995417, test_score:0.7054430236505601
time taken for LogisticRegression: 0:06:21.830647
**********Run Model DecisionTreeClassifier**********
DecisionTreeClassifier -  train_score: 0.6258699711424206, test_score:0.6040511485798348
time taken for DecisionTreeClassifier: 0:00:22.427599
**********Run Model RandomForestClassifier**********
RandomForestClassifier -  train_score: 0.5671928931137894, test_score:0.559465882086681
time taken for RandomForestClassifier: 0:00:11.974119
**********Run Model KNeighborsClassifier**********
KNeighborsClassifier -  train_score: 0.714338256096871, test_score:0.5551657802421637
time taken for KNeighborsClassifier: 0:08:41.905723
**********Run Model LGBMClassifier**********


/opt/conda/anaconda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/anaconda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/anaconda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/anaconda

LGBMClassifier -  train_score: 0.6638657839642392, test_score:0.6430915469050583
time taken for LGBMClassifier: 0:00:09.138587
**********Run Model XGBClassifier**********


/opt/conda/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/conda/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

XGBClassifier -  train_score: 0.697844169071465, test_score:0.6774923616611972
time taken for XGBClassifier: 0:01:30.874204


/opt/conda/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,model_name,model_object,cv_train,train_accuray_score,test_accuray_score
0,LogisticRegression,"LogisticRegression(C=1.0, class_weight=None, d...",0.704804,0.807163,0.705443
0,XGBClassifier,"XGBClassifier(base_score=0.5, booster=None, co...",0.682794,0.697844,0.677492
0,LGBMClassifier,"LGBMClassifier(boosting_type='gbdt', class_wei...",0.658321,0.663866,0.643092
0,DecisionTreeClassifier,"DecisionTreeClassifier(class_weight=None, crit...",0.612262,0.625870,0.604051
0,RandomForestClassifier,"(DecisionTreeClassifier(class_weight=None, cri...",0.567193,0.567193,0.559466
0,KNeighborsClassifier,"KNeighborsClassifier(algorithm='auto', leaf_si...",0.556583,0.714338,0.555166


CPU times: user 32min 49s, sys: 54.1 s, total: 33min 43s
Wall time: 17min 18s


In [20]:
%%time
from sklearn.externals import joblib
best_model = df_outcome.at[0, 'model_object'][0]
joblib.dump(best_model,  'best_model.model')

CPU times: user 3.93 ms, sys: 0 ns, total: 3.93 ms
Wall time: 2.86 ms


['best_model.model']

In [ ]:
%%time
df_test = pd.read_csv('gs://dataproc-6ca41800-27b4-47d5-abee-55c011dfa389-asia-southeast1/data/kaggle/test.csv')
df_test_X = pipeline.transform(df_test['Comment'])
best_model = joblib.load('best_model.model')
df_test['Outcome'] = best_model.predict(df_test_X)
test_result = df_test[['Id', 'Outcome']]
test_result.to_csv('submission.csv', index=False)

CPU times: user 6min 49s, sys: 54 s, total: 7min 43s
Wall time: 7min 43s


In [ ]:
%%time
storage_client = storage.Client()
bucket = storage_client.get_bucket('dataproc-6ca41800-27b4-47d5-abee-55c011dfa389-asia-southeast1')

def upload_files(bucket, files):
    for file in files:
        butcketFile = 'data/kaggle/' + file
        blob = bucket.blob(butcketFile)
        blob.upload_from_filename(file)
        print("Upload {0} to {1}".format(file, butcketFile))
    
upload_files(bucket, ['best_model.model','submission.csv'])

Upload best_model.model to data/kaggle/best_model.model
Upload submission.csv to data/kaggle/submission.csv
CPU times: user 32.4 ms, sys: 4.73 ms, total: 37.1 ms
Wall time: 305 ms
